In [ ]:
# STEP 1 – Install Required Libraries

!pip install tensorflow numpy matplotlib scikit-learn

In [ ]:
# STEP 2 – Import Libraries

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
import requests
from zipfile import ZipFile
import pandas as pd

In [ ]:
# STEP 3 – Download Dataset from GitHub

# The Fruits-360 GitHub repo stores images organized into folders.
# Example for Fruits-360 100x100:
# GitHub repo link: https://github.com/fruits-360/fruits-360-100x100
# Since they host zipped folders, you can download the zip directly:

dataset_url = 'https://github.com/fruits-360/fruits-360-100x100/archive/refs/heads/master.zip'
output_zip = 'fruits-360.zip'

# Download zip
r = requests.get(dataset_url)
with open(output_zip, 'wb') as f:
    f.write(r.content)

# Extract
with ZipFile(output_zip, 'r') as zip_ref:
    zip_ref.extractall('fruits-360')

# Now your folder structure should be:
# fruits-360/fruits-360-100x100-master/fruits-360_100x100/Training/
# fruits-360/fruits-360-100x100-master/fruits-360_100x100/Test/

In [ ]:
# STEP 4 – Prepare Directories

# Combine Training and Test images into one folder for training on 100% data. Here’s how to merge:

base_path = 'fruits-360/fruits-360-100x100-master/fruits-360_100x100/'

train_path = os.path.join(base_path, 'Training')
test_path = os.path.join(base_path, 'Test')

merged_path = 'fruits-merged'

# Create merged directory
if not os.path.exists(merged_path):
    os.makedirs(merged_path)

# Copy all train folders
for class_folder in os.listdir(train_path):
    src = os.path.join(train_path, class_folder)
    dst = os.path.join(merged_path, class_folder)
    shutil.copytree(src, dst)

# Copy all test folders (add images to existing folders if any)
for class_folder in os.listdir(test_path):
    src = os.path.join(test_path, class_folder)
    dst = os.path.join(merged_path, class_folder)
    if os.path.exists(dst):
        # Copy images into existing folder
        for file in os.listdir(src):
            shutil.copy2(os.path.join(src, file), dst)
    else:
        shutil.copytree(src, dst)

print("Data merged!")

In [ ]:
# STEP 5 – Create Data Generators

# Split data into 80/20 random split using ImageDataGenerator with validation_split.

IMG_SIZE = (100, 100)
BATCH_SIZE = 64

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.20
)

train_gen = datagen.flow_from_directory(
    merged_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    subset='training'
)

val_gen = datagen.flow_from_directory(
    merged_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    subset='validation'
)

In [ ]:
# STEP 6 – Build CNN Model

# Simple yet powerful architecture for fruit classification:

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(100, 100, 3)),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(train_gen.num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
# STEP 7 – Train the Model

# Train on 80% → validate on 20%:

history = model.fit(
    train_gen,
    epochs=20,
    validation_data=val_gen
)

In [ ]:
# STEP 8 – Evaluate

# Plot performance:

plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.legend()
plt.show()

In [ ]:
# STEP 9 – Save Model

model.save("fruit_classifier_model.h5")

In [ ]:
# STEP 10 – Load Class Labels

# Store class names for predictions:

class_indices = train_gen.class_indices
# Inverse mapping: index → class name
idx_to_class = {v: k for k, v in class_indices.items()}

In [ ]:
# STEP 11 – Predict New Image

# Upload an image file:
from tensorflow.keras.preprocessing import image

def predict_image(img_path):
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    predictions = model.predict(img_array)
    predicted_class_index = np.argmax(predictions[0])
    predicted_class = idx_to_class[predicted_class_index]
    
    return predicted_class

# Example usage:
img_path = 'your_test_image.jpg'
predicted_fruit = predict_image(img_path)
print(f"Predicted Fruit: {predicted_fruit}")

In [ ]:
# STEP 12 – Get Details About the Fruit

# The Fruits-360 repo has a meta branch: Meta repo

# Download their CSV for fruit attributes:

meta_url = "https://raw.githubusercontent.com/fruits-360/fruits-360-meta/master/fruits-360_dataset_meta.csv"
meta_df = pd.read_csv(meta_url)

# Display details for predicted fruit
def fruit_details(fruit_name):
    result = meta_df[meta_df['Fruit'] == fruit_name]
    if not result.empty:
        return result.to_dict(orient='records')[0]
    else:
        return f"No details found for {fruit_name}"

# Example:
details = fruit_details(predicted_fruit)
print(details)